In [1]:
import os
import pycardano as pc
import random
import time
from marketplace import *
import pathlib

In [2]:
# Copy your BlockFrost project ID below. Go to https://blockfrost.io/ for more information.
# BLOCK_FROST_PROJECT_ID = "your_project_id"
NETWORK = pc.Network.TESTNET

# Use ogmios backend for chain context
context = pc.OgmiosChainContext(ws_url="wss://ogmios-preprod-api-testing-2fdd06.us1.demeter.run", network=NETWORK)

# chain_context = BlockFrostChainContext(
#     project_id=BLOCK_FROST_PROJECT_ID,
#     base_url=ApiUrls.preprod.value,
# )

"""Preparation"""
# Define the root directory where images and keys will be stored.
PROJECT_ROOT = "marketplace"
root = pathlib.Path(PROJECT_ROOT)

# Create the directory if it doesn't exist
root.mkdir(parents=True, exist_ok=True)

"""Generate keys"""
key_dir = root / "keys"
key_dir.mkdir(exist_ok=True)

# Load payment keys or create them if they don't exist
def load_or_create_key_pair(base_dir, base_name):
    skey_path = base_dir / f"{base_name}.skey"
    vkey_path = base_dir / f"{base_name}.vkey"
    stake_skey_path = base_dir / f"{base_name}_stake.skey"
    stake_vkey_path = base_dir / f"{base_name}_stake.vkey"

    if skey_path.exists():
        skey = pc.PaymentSigningKey.load(str(skey_path))
        vkey = pc.PaymentVerificationKey.from_signing_key(skey)
        stake_skey = pc.StakeSigningKey.load(str(stake_skey_path))
        stake_vkey = pc.StakeVerificationKey.from_signing_key(stake_skey)
    else:
        key_pair = pc.PaymentKeyPair.generate()
        key_pair.signing_key.save(str(skey_path))
        key_pair.verification_key.save(str(vkey_path))
        stake_key_pair = pc.StakeKeyPair.generate()
        stake_key_pair.signing_key.save(str(stake_skey_path))
        stake_key_pair.verification_key.save(str(stake_vkey_path))
        stake_skey = stake_key_pair.signing_key
        stake_vkey = stake_key_pair.verification_key
        skey = key_pair.signing_key
        vkey = key_pair.verification_key
    
    return skey, vkey, stake_skey, stake_vkey

In [3]:
owner_skey, owner_vkey, owner_stake_skey, owner_stake_vkey = load_or_create_key_pair(key_dir, "owner")


bidder_skey, bidder_vkey, bidder_stake_skey, bidder_stake_vkey = load_or_create_key_pair(key_dir, "bidder")

In [4]:
owner_address = pc.Address(payment_part=owner_vkey.hash(), staking_part=owner_stake_vkey.hash(), network=NETWORK)
bidder_address = pc.Address(payment_part=bidder_vkey.hash(), staking_part=bidder_stake_vkey.hash(), network=NETWORK)

In [5]:
print(f"Owner address: {owner_address}")
print(f"Bidder address: {bidder_address}")

Owner address: addr_test1qq5g0m0h7kzts8mfuckeuyr2qxk9fgfskz47kx0fz7l45m35sz43s8xuc9qt8kdxlchhy90d4yp0sa43vdrxuu9jewps34ue95
Bidder address: addr_test1qzjr9whvuyldxf9ft6998jjwnq72syr2yeqlmygxfhl2d2c8lxhg9ar5qfnk2w5j0gj6fvygkdcdgd9e7gftjgt4dvhqv0ej3s


In [6]:
script_path = pathlib.Path().cwd()
script_path = script_path.joinpath("build","marketplace", "script.cbor")

In [7]:
with open(script_path) as f:
        cbor_hex = f.read()

cbor = bytes.fromhex(cbor_hex)

In [8]:
plutus_script = pc.PlutusV2Script(cbor)
script_hash = pc.plutus_script_hash(plutus_script)
script_address = pc.Address(script_hash, network=NETWORK)

In [13]:
context.utxos(str(script_address))

[{'input': {
   'index': 0,
   'transaction_id': TransactionId(hex='09a87ffaab78b8ba90c240650af4662cbdb8543b7731a73b54b487314e929479'),
 },
  'output': {
   'address': addr_test1wqvfw4e97r4rw7ue4t47326kuendl8eua4acllfa4t5djgqvzrycq,
   'amount': {'coin': 3000000, 'multi_asset': {}},
   'datum': RawCBOR(cbor=b'\xd8y\x9f\x1a\x05\xf5\xe1\x00X9\x00(\x87\xed\xf7\xf5\x84\xb8\x1fi\xe6-\x9e\x10j\x01\xacT\xa10\xb0\xab\xeb\x19\xe9\x17\xbfZn4\x80\xab\x18\x1c\xdc\xc1@\xb3\xd9\xa6\xfe/r\x15\xed\xa9\x02\xf8v\xb1cFnp\xb2\xcb\x83X\x1c\xa42\xba\xec\xe1>\xd3$\xa9^\x8aS\xcaN\x98<\xa8\x10j&A\xfd\x91\x06M\xfe\xa6\xab\xff'),
   'datum_hash': None,
   'post_alonzo': False,
   'script': None,
 }}]

In [42]:
isinstance(context.utxos(str(script_address))[0], pc.UTxO)

True

In [11]:
# Get payment address    
payment_address = bidder_address

# Find a script UTxO
utxo_to_spend = None
for utxo in context.utxos(str(script_address)):
    if utxo.output.datum:

        try:
                params = Listing.from_cbor(utxo.output.datum.cbor)
                print(params)
        except Exception:
                continue
        
        if (params.owner == bytes(payment_address.payment_part)):
                utxo_to_spend = utxo
                break
assert isinstance(utxo_to_spend, pc.UTxO), "No script UTxOs found!"


AssertionError: No script UTxOs found!

In [32]:
params

NameError: name 'params' is not defined